In [1]:
%load_ext autoreload
%autoreload 2

from scripts import hdc_scraping
from IPython.display import clear_output

In [2]:
# Define what you need to extract
indicator = 'Home visit palliative care'
url = 'https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=b08560518ca0ebcaf2016dab69fb38b5&id=8eb4ec8a40c080aadfdbb895be74e771'
level = 'รายอำเภอ' # Select from ['รายอำเภอ']
yearback = 5

In [4]:
hdc_scraping.scraping(indicator, url, level, yearback)

clear_output()
print("Scraped successfully")

Extracting Home visit palliative care: https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=b08560518ca0ebcaf2016dab69fb38b5&id=8eb4ec8a40c080aadfdbb895be74e771
เขตสุขภาพที่ 1
done
เขตสุขภาพที่ 2
Error: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.53)
Stacktrace:
0   chromedriver                        0x000000010117794c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x000000010116ffa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x0000000100cd5ff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000100caf9c8 chromedriver + 129480
4   chromedriver                        0x0000000100d44ec4 cxxbridge1$string$len + 547704
5   chromedriver                        0x0000000100d5da14 cxxbridge1$string$len + 648904
6   chromedriver                        0x0000000100d11004 cxxbridge1$string$len + 335032
7   chromedriver             

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.53)
Stacktrace:
0   chromedriver                        0x000000010117794c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x000000010116ffa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x0000000100cd5ff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000100caf9c8 chromedriver + 129480
4   chromedriver                        0x0000000100d44ec4 cxxbridge1$string$len + 547704
5   chromedriver                        0x0000000100d5da14 cxxbridge1$string$len + 648904
6   chromedriver                        0x0000000100d11004 cxxbridge1$string$len + 335032
7   chromedriver                        0x00000001011401f4 cxxbridge1$str$ptr + 2501068
8   chromedriver                        0x0000000101143508 cxxbridge1$str$ptr + 2514144
9   chromedriver                        0